In [11]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from IPython.display import clear_output, display
from selenium.webdriver.common.by import By
from selenium import webdriver
from time import sleep
from tqdm import tqdm
import pandas as pd
import re

In [16]:
df = pd.read_csv("links.csv")
df

,youtube_link,caption,download_link,success,downloaded
0,https://youtu.be/EYr7tJCf8C4,1__تئوری_آشوب_به_زبان_ساده_جلسه_۱_چارچوب_فیزیک...,https://srv5.dlmate.cc/dl?hash=VagZEOG6oZ3Vi5B...,True,NaN
1,https://youtu.be/5_rSxy7uk-0,2__تئوری_آشوب_به_زبان_ساده_جلسه_۱_چارچوب_فیزیک...,https://srv4.dlmate.cc/dl?hash=bzk%2FQvyCMB1Lm...,True,NaN
2,https://youtu.be/rn_WmM45cGc,3__تئوری_آشوب_به_زبان_ساده_جلسه_۲_اجزا_و_رابطه...,https://srv2.dlmate.cc/dl?hash=9%2Fl%2B9Dbam7N...,True,NaN
3,https://youtu.be/xH1BOnD0tOE,4__تئوری_آشوب_به_زبان_ساده_جلسه_۳_بودن_و_شدن_ق...,https://srv5.dlmate.cc/dl?hash=Wpo%2BjblUzAzuB...,True,NaN
4,https://youtu.be/tErALPjnCIE,5__تئوری_آشوب_به_زبان_ساده_جلسه_۳_بودن_و_شدن_ق...,https://srv9.dlmate.cc/dl?hash=j0ylAM1ELl%2Fwv...,True,NaN
...,...,...,...,...,...
120,https://youtu.be/NFO1IqNrjIc,121__آشنایی_با_تانسور_و_ریاضیات_تانسوری_جلسه_۵...,https://srv1.dlmate.cc/dl?hash=O5g7zRmqJ8ZL0n4...,True,NaN
121,https://youtu.be/e3xGddJ-HeQ,122__پدیدایی_نگرشی_نو_و_متفاوت_به_مطالعه_انسان...,https://srv4.dlmate.cc/dl?hash=D86zcDETao0RPyJ...,True,NaN
122,https://youtu.be/kry9gbr0w9M,123__پدیدایی_نگرشی_نو_و_متفاوت_به_مطالعه_انسان...,https://srv2.dlmate.cc/dl?hash=I4n%2BRZKb%2Bu5...,True,NaN
123,https://youtu.be/jDgdEayWLnQ,124__پدیدایی_نگرشی_نو_و_متفاوت_به_مطالعه_انسان...,https://srv3.dlmate.cc/dl?hash=jvFR98QNLPHY2ji...,True,NaN


In [13]:
def sanitize_filename(caption):
    # caption = re.sub(r'Duration:\s*\d{1,2}:\d{2}\s*minutes$', '', caption, flags=re.MULTILINE)
    caption = re.sub(r'Duration.*$', '', caption, flags=re.MULTILINE)
    filename = re.sub(r'[^\w\s-]', '', caption)
    filename = re.sub(r'[\s\n]+', '_', filename)
    filename = filename.strip('_')
    return filename[:240]

In [14]:
def save_download_video(youtube_link):
    try:
        url = 'https://en.y2mate.is/v96/'

        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')

        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

        # driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

        driver.get(url)

        driver.maximize_window()

        a = driver.find_elements(By.ID, "txtUrl")[0].send_keys(youtube_link)

        b = driver.find_elements(By.ID, "btnSubmit")[0].click()

        sleep(3)

        trs = driver.find_elements(By.TAG_NAME, 'tr')

        for i in trs:
            if "360p" in i.text or "480p" in i.text or "720p" in i.text:
                tr = i
                break

        tr.find_element(By.TAG_NAME, 'button').click()

        sleep(3)

        count = 0

        while True:

            count += 1

            if count > 20:
                break

            try:
                download_btn = driver.find_element(By.ID, "btnOk")
                break

            except:
                sleep(3)
                continue

        x = download_btn.find_element(By.TAG_NAME, "button").find_element(By.TAG_NAME, "a")

        sleep(3)

        download_link = x.get_attribute("href")

        caption = driver.find_element(By.TAG_NAME, "b").text

        caption = sanitize_filename(caption)

        driver.quit()

        return download_link, caption
    
    except Exception as e:
        print(e)
        return None, None

In [17]:
for index, row in tqdm(df.iterrows()):

    clear_output(wait=True)
    display(f"INDEX : {index}")

    if pd.notna(row['download_link']):
        continue

    download_link, caption = save_download_video(row['youtube_link'])

    caption = f"{index + 1}__{caption}"

    df.at[index, 'download_link'] = download_link
    df.at[index, 'caption'] = caption

    if download_link and caption:
        df.at[index, 'success'] = True

    df.to_csv("links.csv", index=False)

'INDEX : 124'

125it [03:58,  1.91s/it]
